In [ ]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 4.3 MB/s 


In [ ]:
import tensorflow as tf
import keras_tuner as kt
from tensorflow import keras
print(f"TensorFlow Version: {tf.__version__}")
print(f"KerasTuner Version: {kt.__version__}")

TensorFlow Version: 2.8.2
KerasTuner Version: 1.1.3


In [ ]:
# Load and split data into train and test sets
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
# Normalize pixels to values between 0 and 1
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [ ]:
# Build baseline model with Sequential API
b_model = keras.Sequential()
b_model.add(keras.layers.Flatten(input_shape=(28,28)))
b_model.add(keras.layers.Dense(units=512, activation='relu', name='dense_1'))
b_model.add(keras.layers.Dropout(0.2))
b_model.add(keras.layers.Dense(10, activation='softmax'))
# Print model summary
b_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_1 (Dense)             (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Set training parameters
b_model.compile(optimizer=keras.optimizers.Adam(lr=0.001),
loss=keras.losses.SparseCategoricalCrossentropy(),
metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# Number of epochs
NUM_EPOCHS = 20
# Early stopping set after 5 epochs
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
# Train model
b_model.fit(X_train, y_train, epochs=NUM_EPOCHS, validation_split=0.2, callbacks=[stop_early])

Epoch 1/20
1500/1500 [==============================] - 12s 7ms/step - loss: 0.5142 - accuracy: 0.8171 - val_loss: 0.4071 - val_accuracy: 0.8547
Epoch 2/20
1500/1500 [==============================] - 11s 7ms/step - loss: 0.3913 - accuracy: 0.8583 - val_loss: 0.3702 - val_accuracy: 0.8675
Epoch 3/20
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3565 - accuracy: 0.8681 - val_loss: 0.3768 - val_accuracy: 0.8648
Epoch 4/20
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3361 - accuracy: 0.8756 - val_loss: 0.3581 - val_accuracy: 0.8724
Epoch 5/20
1500/1500 [==============================] - 10s 7ms/step - loss: 0.3158 - accuracy: 0.8835 - val_loss: 0.3207 - val_accuracy: 0.8865
Epoch 6/20
1500/1500 [==============================] - 10s 7ms/step - loss: 0.3026 - accuracy: 0.8856 - val_loss: 0.3212 - val_accuracy: 0.8813
Epoch 7/20
1500/1500 [==============================] - 10s 7ms/step - loss: 0.2893 - accuracy: 0.8910 - val_loss: 0.3407 - val_accu

In [ ]:
b_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_1 (Dense)             (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import pandas as pd
def evaluate_model(model, X_test, y_test):
    eval_dict = model.evaluate(X_test, y_test, return_dict=True)
    display_df = pd.DataFrame([eval_dict.values()], columns=[list(eval_dict.keys())])
    return display_df

results = evaluate_model(b_model, X_test, y_test)
results.index = ['Baseline']
results.head()

313/313 [==============================] - 2s 6ms/step - loss: 0.3376 - accuracy: 0.8869


,loss,accuracy
Baseline,0.33758,0.8869


In [ ]:
def build_model(hp):

    # Initialize sequential API and start building model.
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28,28)))
    # Tune the number of hidden layers and units in each.
    # Number of hidden layers: 1 - 5
    # Number of Units: 32 - 512 with stepsize of 32
    for i in range(1, hp.Int("num_layers", 2, 6)):
        model.add(
        keras.layers.Dense(
        units=hp.Int("units_" + str(i), min_value=32, max_value=512, step=32),
        activation="relu")
    )
        # Tune dropout layer with values from 0 - 0.3.
        model.add(keras.layers.Dropout(hp.Float("dropout_" + str(i), 0, 0.3, step=0.1)))
    # Add output layer.
    model.add(keras.layers.Dense(units=10, activation="softmax"))
    # Tune learning rate for Adam optimizer with values from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(),
                metrics=["accuracy"])
    return model

In [ ]:
# Instantiate the tuner
tuner = kt.Hyperband(build_model,
objective="val_accuracy",
max_epochs=20,
factor=3,
hyperband_iterations=1,
directory="kt_dir",
project_name="kt_hyperband",
overwrite=True)

In [ ]:
# Display search space summary
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 6, 'step': 1, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
dropout_1 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.3, 'step': 0.1, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [ ]:
# Get the optimal hyperparameters from the results
best_hps=tuner.get_best_hyperparameters()[0]

h_model = tuner.hypermodel.build(best_hps)
h_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_5 (Dense)             (None, 384)               301440    
                                                                 
 dropout_4 (Dropout)         (None, 384)               0         
                                                                 
 dense_6 (Dense)             (None, 352)               135520    
                                                                 
 dropout_5 (Dropout)         (None, 352)               0         
                                                                 
 dense_7 (Dense)             (None, 512)               180736    
                                                                 
 dropout_6 (Dropout)         (None, 512)              

In [ ]:
# Train the hypertuned model
h_model.fit(X_train, y_train, epochs=NUM_EPOCHS, validation_split=0.2, callbacks=[stop_early])

Epoch 1/20
1500/1500 [==============================] - 16s 11ms/step - loss: 0.6560 - accuracy: 0.7695 - val_loss: 0.4461 - val_accuracy: 0.8395
Epoch 2/20
1500/1500 [==============================] - 16s 10ms/step - loss: 0.4279 - accuracy: 0.8465 - val_loss: 0.3894 - val_accuracy: 0.8594
Epoch 3/20
1500/1500 [==============================] - 16s 11ms/step - loss: 0.3727 - accuracy: 0.8651 - val_loss: 0.3465 - val_accuracy: 0.8769
Epoch 4/20
1500/1500 [==============================] - 17s 11ms/step - loss: 0.3418 - accuracy: 0.8752 - val_loss: 0.3386 - val_accuracy: 0.8758
Epoch 5/20
1500/1500 [==============================] - 16s 11ms/step - loss: 0.3180 - accuracy: 0.8829 - val_loss: 0.3305 - val_accuracy: 0.8786
Epoch 6/20
1500/1500 [==============================] - 16s 11ms/step - loss: 0.3007 - accuracy: 0.8886 - val_loss: 0.3123 - val_accuracy: 0.8879
Epoch 7/20
1500/1500 [==============================] - 16s 11ms/step - loss: 0.2842 - accuracy: 0.8955 - val_loss: 0.3102 -

In [ ]:
hyper_df = evaluate_model(h_model, X_test, y_test)
hyper_df.index = ["Hypertuned"]
results.append(hyper_df)

313/313 [==============================] - 2s 5ms/step - loss: 0.3317 - accuracy: 0.8863


,loss,accuracy
Baseline,0.337580,0.8869
Hypertuned,0.331749,0.8863
